\* Keyphrase, candidats 등 문맥을 고려해 '키워드'로 통칭

## ■ KeyBERT 참고 문헌

> [- Keyword Extraction with BERT](https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea)  
> [- BERT 변환기와 명사구를 사용한 핵심구 추출](https://medium.com/towards-data-science/enhancing-keybert-keyword-extraction-results-with-keyphrasevectorizers-3796fa93f4db)  
> [- KeyBERT로 관련 키워드를 추출하는 방법](https://towardsdatascience.com/how-to-extract-relevant-keywords-with-keybert-6e7b3cf889ae)  
> [- KeyBERT Git](https://github.com/MaartenGr/KeyBERT)  
  
---

## ■ KeyBERT 이론 설명

### - 통계 기반 추출 방식? 의미론적 추출 방식!
- 통계 기반 추출 방식에는 RAKE, YAKE!, TF-IDF 방식 등이 있음
- 통계 기반 추출의 경우, 글 전체의 의미가 반드시 고려되진 않음

### - 작동 원리(KeyBert의 작동원리)
1\) 사전 학습된 언어 모델을 정의
- KeyBERT는 임베딩을 위해 기본적으로 BERT를 사용하지만 BERT 기반의 많은 사전 학습 언어 모델을 지원
- 창작자는 당시 senetence-transformer 패키지의 DistilBERT 모델을 사용
- DistilBERT는 기존 BERT보다 좀 더 가볍지만 성능은 그에 준하는 모델 (BERT가 큰 규모의 문서에는 효율적이지 않기에 상대적으로 가벼운 것 사용)

2\) 키워드 및 표현식 추출  
- Bag Of Words(=CountVectorizer) 기술을 사용하여 동일한 문서에 키워드들을 추출  
\* CountVectorizer: 명사구 초점에서 벗어나서 키워드 길이 조정 및 핵심 구문 만들 수 있음, 빠른 stopwords 제거  
\* n_gram_range: 키워드(구문)의 길이 지정 가능, stopwords 제거하지 않으면 키 구문이 길게 나올 수도 있음
  
![](https://miro.medium.com/v2/resize:fit:828/0*Vg6abk6F1H6JfwDB)
<center>1,2) 키워드 및 표현식 추출</center>

<hr style="border: none; border-top: 2px dashed gray;"/>
  
3\) 텍스트 임베딩
- 그림2번 처럼 입력된 키워드들은 사전 학습된 언어 모델을 통해 임베딩 벡터로 변환
- 이때 사전 학습 모델로 임베딩 (본 문서에는 DistilBERT)

![](https://miro.medium.com/v2/resize:fit:786/0*9B-eJJR6p4jb_-uA)
<center>3) 텍스트 임베딩</center>
  
<hr style="border: none; border-top: 2px dashed gray;"/>
  
4\) 키워드 유사도 계산
> [코사인 유사도란? (Cosine Similarity)](https://wikidocs.net/24603)  
- 키워드와 문서가 같은 공간에 표시되므로 KeyBERT는 키워드 임베딩과 문서 임베딩 간의 코사인 유사도를 계산
- 그 이후 문서와 키워드간 유사도가 높은 순서대로 나열됨
- 단, 나열된 키워드들은 n_gram으로 묶었을 때 각 후보별로 의미가 너무 유사하는 문제 발생할 수 있음

![](https://miro.medium.com/v2/resize:fit:786/0*ocfR8WLpwRI_7tGi)
<center>4)코사인 유사도 계산</center>
  
<hr style="border: none; border-top: 2px dashed gray;"/>
  
5\) 키워드 Diversification
(추출된 키워드의 다양성을 올리는 작업)

- Max Sum Similarity (MSS) - 최대 합계 거리
  - 키워드들의 유사도를 고려하여 문서에서 가장 관련성이 높은 키워드들을 선택하는 알고리즘
  - 각 키워드와 문서 간의 코사인 유사도를 계산하고, 문서에서 키워드들의 코사인 유사도 합이 최대가 되도록 키워드들을 선택
  - 즉, 데이터(후보) 쌍 간의 거리가 최대가 되도록 하는 방법  
\* nr_candidate를 늘리면 다양성이 증가하지만 너무 높을 경우 표현력 저하  
\** nr_candidate는 전체 고유 단어 수의 20% 미만을 추천

- Maximal Marginal Relevance (MMR) - 최대최근 관련성
  - MMR은 키워드들의 중복을 최소화하면서 다양한 토픽의 키워드들을 선택하는 알고리즘입니다.
  - 이 알고리즘은 관련성과 다양성 사이의 trade-off를 고려합니다. 
  - 먼저, 가장 관련성이 높은 키워드를 선택하고, 이후 다른 키워드들 중에서 가장 관련성이 높은 키워드들과의 유사도와 중복을 최소화하면서 다양한 키워드들을 선택합니다. 이를 통해 문서의 다양한 측면을 파악할 수 있도록 합니다.
\* EmbedRank 알고리즘 기반
\** diversity 증감으로 조절 가능

---

## ■ Enhanced KeyBERT

### \- ❗임베딩 단계 이전의 문서로부터 키워드를 추출하는 방식에는 문제가 있음
- 최적의 n_gram 범위를 모르므로 적절한 범위를 찾는데 시간↑
- 추출된 키워드의 문맥과 문법 구조가 부정확할 수 있음
  
### \- 해결방법: KeyBERT의 키워드 추출을 KeyphraseVectorizer 패키지와 함께 사용
- 전체 문서에 spaCy 패키지를 활용하여 각 토큰별로 품사 태그를 달고, 정규식을 이용해서 해당하는 구문 추출
- 장점: 사용자가 정의한 Simple n_gram 대신에 문법, 문맥적으로 더 정확한 키 구문을 얻을 수 있음
  
### \- 작동방식 
※ 기본적으로 모든 Parameter가 English에 맞춰져 있음

1\) 전체 문서 텍스트에 토큰별로 spaCy 품사 태그가 달림

2\) 품사 태그가 미리 정의된 정규 표현식 패턴과 일치하는 키워드를 추출  
\* 기본적으로 0 이상의 형용사와 1개 이상의 명사로 구성된 키워드 추출 (기본 정규식 패턴: '<J.*>*<N.*>+')  

3\) 추출된 키워드들을 document-keyphrase matrix로 변환  
\* document-keyphrase matrix: 문서의 모음에서 특정 구문(=키워드)가 얼마나 자주 등장하는지를 나타내는 수학적 행렬
\* 각 열(column)은 개별 구문을, 각 행(row)은 개별 문서를, 그 교차점에 있는 값(value)은 해당 문서에서 해당 구문이 얼마나 자주 등장하는지(즉, 빈도수)를 나타냄

4\) 추출된 키워드는 이후 임베딩 생성 및 유사도 계산을 위해 KeyBERT로 전달  
\* 분산 표현하여 각 키워드 간 의미적 유사성을 벡터화(임베딩) [*분산 표현?](https://wikidocs.net/22660)  

5\) 각 문서와 가장 유사한 키워드 상위 n개를 나열하여 각 문장을 키워드로 묘사



---

## ■ KeyBERT 구현

In [ ]:
# !pip install keybert
# !pip install keyphrase-vectorizers

In [2]:
cd "../../Data/Preprocessed_data"

c:\Users\user\OneDrive - 한국항공대학교\바탕 화면\머신러닝\프로젝트\Bert_beer_sentiment_anlysis\Data\Preprocessed_data


In [3]:
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer # n_gram 대신 문맥, 문법을 고쳐주는 Vectorizer
# from flair.embeddings import TransformerDocumentEmbeddings # 사전학습된 임베딩을 사용하는 라이브러리
import pandas as pd
from pprint import pprint # 출력 정리 라이브러리

In [4]:
df = pd.read_csv('Binary Classification_v4.csv')
df

,Review,Beer_name,MultinomialNB_label
0,This is like Budlight but with slight corn tas...,Milwaukee's Best Light,Negative
1,Strong corn flavor. Highly carbonated and no h...,Milwaukee's Best Light,Negative
2,It just doesn't get worse than this. Brings me...,Milwaukee's Best Light,Negative
3,Beast Bleu. I shutter to think how many cans ...,Milwaukee's Best Light,Negative
4,I wouldn't wish this beer on anyones glass. T...,Milwaukee's Best Light,Negative
...,...,...,...
94136,"Can (Jan 8, 2020 canning).\n\nHead is initiall...",The Bruery Ruekeller Helles,Positive
94137,Pale clear gold with thin white head. Light h...,The Bruery Ruekeller Helles,Positive
94138,Can. Pours brilliantly clear gold with a mediu...,The Bruery Ruekeller Helles,Positive
94139,"Taster at The Brewpub, Placentia, Ca. Golden w...",The Bruery Ruekeller Helles,Positive


In [8]:
beer_names = df['Beer_name'].unique()
review_counts = df.groupby('Beer_name').size()
# print(beer_names)
review_counts.head(10)

Beer_name
8 Wired iStout                      402
ARK Seoulite Ale                      7
AleSmith Speedway Stout            1414
Asahi Super Dry                    1691
Asahi Super Dry Black                94
B-40 Bull Max 8%                     21
Bavaria 8.6 (Original)              274
Bavaria Pilsener / Premium Beer     792
Beck's                             2062
Beer 30 Ice                          49
dtype: int64

In [18]:
############## 지정된 맥주 리뷰 df 분리 ##############
df_f = df[df['Beer_name'] == "Founders Porter"]
df_f.reset_index(drop = True,inplace = True)

,Review,Beer_name,MultinomialNB_label
0,Surprisingly little taste. It is fresh and a t...,Asahi Super Dry,Positive
1,Pours an almost honey color with an extremely ...,Asahi Super Dry,Positive
2,"Smell and taste of maltiness, grass and bread....",Asahi Super Dry,Positive
3,No redeeming features of this 'beer' other tha...,Asahi Super Dry,Negative
4,"At least, there's no harsh/off-flavours. What'...",Asahi Super Dry,Negative
...,...,...,...
2100,shrewd metal piece of work and a aroma. with d...,Asahi Super Dry,Negative
2101,freak color. it. equal compose Canadian-Molson...,Asahi Super Dry,Negative
2102,middling japanese beer weak/watery. suck.........,Asahi Super Dry,Negative
2103,"character. only thirsty. designate smell, big ...",Asahi Super Dry,Negative


In [28]:
# # 문서 지정
# doc = df['Review'].iloc[0]
# print(doc)

# # 지정된 맥주 리뷰 문서 지정
# doc = df_f['Review'].iloc[1]
# print(doc)

doc = "Not treated before for some reason. Pale, clear blond. Grassy aroma. Taste of straw and some malt. Low bitterness."

In [19]:
# Init KeyBERT: "all-mpnet-base-v2" is best Pretrained Model
kw_model = KeyBERT("all-mpnet-base-v2")

# You can select any model from sentence-transformers [here](https://www.sbert.net/docs/pretrained_models.html) 
# pre-trained BERT-based models: they have shown great performance in semantic similarity and paraphrase identification respectively
# kw_model = KeyBERT("distilbert-base-nli-stsb-mean-tokens")
# kw_model = KeyBERT("xlm-r-distilroberta-base-paraphase-v1")

In [30]:
# KeyphraseCountVectorizer 기본
keywords = kw_model.extract_keywords(doc, vectorizer=KeyphraseCountVectorizer())
pprint(keywords)

[('grassy aroma', 0.5045),
 ('malt', 0.4073),
 ('clear blond', 0.3822),
 ('low bitterness', 0.3635),
 ('straw', 0.3272)]


In [10]:
# KeyphraseCountVectorizer + 최대 합계 거리
keywords = kw_model.extract_keywords(doc, vectorizer=KeyphraseCountVectorizer(), stop_words='english',
                              use_maxsum=True, nr_candidates=20, top_n=5)
print(*keywords, sep="\n")

('front', 0.0834)
('full body', 0.103)
('hint', 0.1218)
('soft carbonation', 0.3384)
('roasted malts', 0.4614)


In [11]:
# KeyphraseCountVectorizer + 최대최근 관련성
keywords = kw_model.extract_keywords(doc, vectorizer=KeyphraseCountVectorizer(), stop_words='english',
                           use_mmr=True, diversity=0.7, top_n=5)
print(*keywords, sep="\n")

('toffee aroma', 0.529)
('caramel malts', 0.4739)
('tan head', 0.4137)
('prominent taste', 0.3748)
('soft carbonation', 0.3384)


In [33]:
# 최대 합계 거리
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words='english',
                              use_maxsum=True, nr_candidates=20, top_n=3, highlight=True)
pprint(keywords)

Not treated before for some reason Pale clear blond Grassy aroma Taste of straw and some malt Low bitterness

[('treated reason pale', 0.3347),
 ('clear blond', 0.3822),
 ('grassy aroma taste', 0.5152)]


In [34]:
# 최대최근 관련성
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words='english',
                           use_mmr=True, diversity=0.7, highlight=True, top_n=3)
pprint(keywords)

Not treated before for for some reason Pale clear blond Grassy aroma of straw and some malt Low bitterness

[('blond grassy aroma', 0.6178),
 ('malt low bitterness', 0.5016),
 ('treated', 0.2173)]


### - 특정 맥주에 대한 리뷰 키워드 추출

In [21]:
cd "./Beer_Sentiment_analysis/Data"

[WinError 3] 지정된 경로를 찾을 수 없습니다: './Beer_Sentiment_analysis/Data'
c:\Users\Gyeom\OneDrive - 한국항공대학교\바탕 화면\머신러닝\프로젝트\맥주 측면 감정 분석\Beer_Sentiment_analysis\Data


In [22]:
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer # n_gram 대신 문맥, 문법을 고쳐주는 Vectorizer
import pandas as pd
from pprint import pprint # 출력 정리 라이브러리

In [23]:
import os
from tensorflow.python.client import device_lib
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 721756377110786641
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6284115968
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6254377679399951221
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [25]:
def extract_keywords_for_beer(doc):
    try:
        # 방법 1
        keywords1 = kw_model.extract_keywords(doc, vectorizer=KeyphraseCountVectorizer(), top_n=5)
        keywords1 = [keyword for keyword, score in keywords1]

        # 방법 2
        keywords2 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words="english",
                                use_maxsum=True, nr_candidates=20, top_n=3)
        keywords2 = [keyword for keyword, score in keywords2]

        # 방법 3
        keywords3 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words="english",
                                use_mmr=True, diversity=0.7, top_n=3)
        keywords3 = [keyword for keyword, score in keywords3]

        # 결과를 하나의 리스트로 합치기
        keywords_combined = list(set(keywords1 + keywords2 + keywords3))

    except ValueError:
        # 키워드를 찾지 못한 경우 빈 리스트 반환
        keywords_combined = []

    return keywords_combined

# 데이터 불러오기
df = pd.read_csv('Binary Classification_v3.csv')


############## 지정된 맥주 이름의 리뷰들 필터링 ##############
beer_names = ['Asahi Super Dry', '8 Wired iStout', 'Red Rock']

# 해당하는 맥주 이름을 가진 리뷰들만 선택하기
selected_reviews = df[df['Beer_name'].isin(beer_names)]
selected_reviews.reset_index(drop = True,inplace = True)
##########################################################

# KeyBERT 모델 생성
kw_model = KeyBERT("all-mpnet-base-v2")


# 키워드 추출 함수 적용
selected_reviews['Keywords'] = selected_reviews['Review'].apply(extract_keywords_for_beer)

# 결과 출력
selected_reviews


In [12]:
selected_reviews.to_csv('pp_ke_selected_reviews.csv', encoding='utf-8', index = False)